In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from glob import glob
import sys
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import seaborn as sns

import tensorflow.keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, BatchNormalization

In [ ]:
train_path = '../input/paddy-disease-classification/train_images'
test_path = '../input/paddy-disease-classification/test_images'

In [ ]:
image_count = len(list(pathlib.Path(train_path).glob('*/*.jpg')))
print(image_count)

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    # shuffle = False,
    seed=12,
    image_size=(448, 448),
    batch_size=16)

In [ ]:
val_set = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
#     shuffle = False,
    seed=12,
    image_size=(448, 448),
    batch_size=16)

In [ ]:
class_names = training_set.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20))
for images, labels in training_set.take(1):
    for i in range(15):
        ax = plt.subplot(5, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
IMAGE_SIZE = [448, 448]
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(inception.output)

In [ ]:
folders = glob(train_path+'/*')

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=inception.input, outputs=prediction)

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_ds = train_datagen.flow_from_directory(train_path,
                                             target_size = (448, 448),
                                             batch_size = 16,
                                             class_mode = 'categorical',
                                             subset='training',
                                             shuffle = False)

In [ ]:
val_ds = train_datagen.flow_from_directory(train_path,
                                             target_size = (448, 448),
                                             batch_size = 16,
                                             class_mode = 'categorical',
                                             subset='validation',
                                             shuffle = False)

In [ ]:
%%time
tf.config.run_functions_eagerly(True)
r = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs=50,
)

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
probabilities = model.predict_generator(val_ds)
y_pred = np.argmax(probabilities, axis=1)

print(confusion_matrix(val_ds.classes, y_pred))

In [ ]:
cf_matrix = confusion_matrix(val_ds.classes, y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(cf_matrix, annot=True)

In [ ]:
print(classification_report(val_ds.classes, y_pred))

In [ ]:
test_path = '../input/paddy-disease-classification/test_images'
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255).flow_from_directory(    
    directory=test_path,
    target_size=(448, 448),
    batch_size=16,
    classes=['.'],
    shuffle=False,
)

In [ ]:
predict = model.predict(test_gen, verbose=1)

In [ ]:
predicted_class_indices=np.argmax(predict,axis=1)
print(set(predicted_class_indices))

inv_map = {v:k for k,v in train_ds.class_indices.items()}
predictions = [inv_map[k] for k in predicted_class_indices]

In [ ]:
filenames=test_gen.filenames

results=pd.DataFrame({"image_id":filenames,
                      "label":predictions})
results.image_id = results.image_id.str.replace('./', '')
results.to_csv("submissioninseption3-4.csv",index=False)
results

In [ ]:
results['label'].value_counts()